In [2]:
import numpy as np
import random
import pickle

In [3]:
# loading training set features
f = open("Datasets/train_set_features.pkl", "rb")
train_set_features2 = pickle.load(f)
f.close()

# reducing feature vector length 
features_STDs = np.std(a=train_set_features2, axis=0)
train_set_features = train_set_features2[:, features_STDs > 52.3]

# changing the range of data between 0 and 1
train_set_features = np.divide(train_set_features, train_set_features.max())

# loading training set labels
f = open("Datasets/train_set_labels.pkl", "rb")
train_set_labels = pickle.load(f)
f.close()

# ------------
# loading test set features
f = open("Datasets/test_set_features.pkl", "rb")
test_set_features2 = pickle.load(f)
f.close()

# reducing feature vector length 
features_STDs = np.std(a=test_set_features2, axis=0)
test_set_features = test_set_features2[:, features_STDs > 48]

# changing the range of data between 0 and 1
test_set_features = np.divide(test_set_features, test_set_features.max())

# loading test set labels
f = open("Datasets/test_set_labels.pkl", "rb")
test_set_labels = pickle.load(f)
f.close()

# ------------
# preparing our training and test sets - joining datasets and lables
train_set = []
test_set = []

for i in range(len(train_set_features)):
    label = np.array([0,0,0,0])
    label[int(train_set_labels[i])] = 1
    label = label.reshape(4,1)
    train_set.append((train_set_features[i].reshape(102,1), label))
    

for i in range(len(test_set_features)):
    label = np.array([0,0,0,0])
    label[int(test_set_labels[i])] = 1
    label = label.reshape(4,1)
    test_set.append((test_set_features[i].reshape(102,1), label))

# shuffle
random.shuffle(train_set)
random.shuffle(test_set)

In [4]:
def sigmoid(z):
    return 1/(1 + np.exp(-z, dtype=np.longdouble))

def d_sigmoid(z):
    return sigmoid(z) * (1 - sigmoid(z))

In [5]:
class MLP:
    
    weights: list[np.ndarray]
    biases: list[np.ndarray]
    
    def __init__(self, layers_size: list[tuple[int, int]]):
        self.weights = []
        self.biases = []
        self.learning_rate = 1
        
        for size in layers_size:
            self.weights.append(np.random.normal(size=size))
            self.biases.append(np.zeros((1, size[1])).astype(np.longdouble))
        
        self.layers_number = len(self.weights)

    def feed(self, data):
        W = self.weights
        b = self.biases
        
        result = []
        o = sigmoid(W[0].T.dot(data) + b[0].T)
        result.append(o)
        for i in range(1, self.layers_number):
            o = sigmoid(W[i].T.dot(o) + b[i].T)
            result.append(o)
        return result
    
    def train(self, x, y, batch_size=10, epoch_number=5, validation=0.2):
        w = self.weights
        b = self.biases
        
        for i in range(epoch_number):
            for j in range(0, x.shape[1], batch_size):
                data = x[:, j:j+batch_size]       # A batch of inputs
                y_ex = y[:, j:j+batch_size]       # Expected Output
                out = self.feed(data)             # Outputs of each layer for the given batch
                
                # Back Propagation Phase
                e2 = (out[2] - y_ex) * d_sigmoid(out[2])
                e1 = w[2].dot(e2) * d_sigmoid(out[1])
                e0 = w[1].dot(e1) * d_sigmoid(out[0])
                
                w[0] -= self.learning_rate * (1/batch_size) * data.dot(e0.T)
                w[1] -= self.learning_rate * (1/batch_size) * out[0].dot(e1.T)
                w[2] -= self.learning_rate * (1/batch_size) * out[1].dot(e2.T)
                
                b[0] -= self.learning_rate * (1/batch_size) * np.sum(e0.T, axis=0)
                b[1] -= self.learning_rate * (1/batch_size) * np.sum(e1.T, axis=0)
                b[2] -= self.learning_rate * (1/batch_size) * np.sum(e2.T, axis=0)
                
    def test(self, x, y):
        out = self.feed(x)
        maximum = np.argmax(out[-1], axis=0)
        equality = (maximum == np.argmax(y, axis=0)).astype('int')
        return sum(equality) / y.shape[1]


In [6]:
# Seperate x and y of train set
x_train = np.column_stack([i[0] for i in train_set])
y_train = np.column_stack([i[1] for i in train_set])

# Seperate x and y of test set
x_test = np.column_stack([i[0] for i in test_set])
y_test = np.column_stack([i[1] for i in test_set])

# Train by 200 items

Here we train our model by 200 items of train set and finally we will witness that it has accuracy around 0.95.

In [8]:
# Create neural network object
mlp = MLP([(102, 150), (150, 60), (60, 4)])

# Using 200 items of test set to see the accuracy for Phase 1
print('Accuracy before train:', mlp.test(x_test[:, :200], y_test[:, :200]))

# Training the model
mlp.train(x_train[:, :200], y_train[:, :200])

# Using whole test set to see the accuracy
print('Accuracy after train:', mlp.test(x_test, y_test))

Accuracy before train: 0.21
Accuracy after train: 0.9833836858006042


# Train and Test with whole datasets

In this part first we make a numpy array of the train_set and test_set. After that we seperate 200 items of test set to see the accuray of model. It is close to 0.25 normally. Then we train an we will see the accuracy rises up to 1.0.

In [ ]:
# Create neural network object
mlp = MLP([(102, 150), (150, 60), (60, 4)])

# Using 200 items of test set to see the accuracy for Phase 1
print('Accuracy before train:', mlp.test(x_test[:, :200], y_test[:, :200]))

# Training the model
mlp.train(x_train, y_train)

# Using whole test set to see the accuracy
print('Accuracy after train:', mlp.test(x_test, y_test))

In [10]:
np.split(np.array[[1, 1, 1, 1, 1]], [0.2* 5])

TypeError: 'builtin_function_or_method' object is not subscriptable